In [173]:
# Importing Dependencies

import numpy as np
import pandas as pd
import importlib # to refresh the cleaning_helper file
import cleaning_helper
importlib.reload(cleaning_helper)
from cleaning_helper import expanding_car_name_col,MPG_mean_calculation,fuel_map,drivetrain_map,speeds_in_transmission,extracting_transmission,expending_convenience,expanding_entertainment,expanding_exterior,expanding_safety,expanding_seating,expanding_seller_address,expanding_mpge,expanding_battery_range_score
import re

In [160]:
#  for better visibility of cols
pd.set_option("display.max_columns", None)

# reading the data
df = pd.read_parquet('C:/Users/aksha/OneDrive/Desktop/CARS-FINAL_YEAR_PROJECT/CLEANING/POST_CLEANING_1_DATA_PART_5.parquet',engine='fastparquet')

#### ISSUES WITH DATASET

1. `New_used` : Name of this col should be changed to `Stock Type`
2. `Car Name` : Car_name col contains info of manufacture Year, and some detailed name of car, which should be splitted into diff cols.
3. `Mileage` : suffix mi. should be removed so that we can convert mileage to number.
4. `Price` : Since all cars are listed in USD,the suffix '$' should be removed and widespace should also be removed, so that we can convert it to numeric format.
5. `Price Drop` : Widespace and suffix price drop should be removed.
6. `Deal_type` : Sometimes contains a price (which is EMI val), which should be splitted into two cols and widespace need to be removed.
7. `Key_specs`,`Basics`,`Other_features`,`Vehicle_history` : These colunm need to be removed, because we expanded this col, but kept if need in future, after all the cleaning process.
8. `Exterior_color` : This col sometimes contains color code (numbers) which need to be converted back to color name.
9. `Interior_color` : col sometimes contains '-' to show null value and color code that need to be converted back to color name.
10. `Drivetrain` : suffix '-wheel Drive' & widespaces should be removed.
11. `MPG` : contains unnecessary text. 
12. `Fuel_Type` : contains '-' to show null values.
13. `Transmission` : need to remove widespaces.
14. `Engine` : contains some specific info about the engine, need to be cleaned with domain knowledge.
15. `VIN`,`Stock_no`,`Accidents_or_damage`,`Clean_title`,`one_owner_vehicle`,`Personal_use_only`,`Open_recall`,`seller_name`,`seller_rating`,`seller_site`,`car_rating`, : seems good.
16. `Mileage` : it is duplicate col and need to be deleted.
17. `Convenience`,`Entertainment`,`Exterior`,`Safety`,`Seating` : contains multiple info need to further splitted into more cols.
18. `Fuel_Type_e` : contains only one value which is electric, hence need to be merged with Fuel_Type col.
19. `mpge` : is used only in electric cars, will see after domain knowledge whether we can merge it with mpg col.
20. `level2_charging`,`dc_fast_charging` : suffix hrs and widespace need to be removed.
21. `battery_capacity` : suffix kWh & widespaces need to be removed.
22. `excepted_range` : need to remove mi and widespace and with domain knowledge, will try to convert hyphened value a single number.
23. `battery_range_score` : contains 2 info, need to be splitted into 2 cols
24. `people_count_seller_rating`,`people_count_car_rating` : need to remove widespace, brackets and suffix reviews.
25. `seller_address` : need to be splitted into city, state, ZIP code and shop number.

In [161]:
# data overview
df.sample(5)

,image_list,new_used,car_name,mileage,price,price_drop,deal_type,key_specs,Exterior_color,Interior_color,Drivetrain,MPG,Fuel_Type,Transmission,Engine,VIN,Stock_no,Mileage,Convenience,Entertainment,Exterior,Safety,Seating,Accidents_or_damage,Clean_title,one_owner_vehicle,Personal_use_only,Open_recall,Fuel_Type_e,mpge,level2_charging,dc_fast_charging,battery_capacity,excepted_range,battery_range_score,basics,features,other_features,vehicle_history,seller_name,seller_rating,people_count_seller_rating,seller_address,seller_site,car_rating,people_count_car_rating
13607,['https://platform.cstatic-images.com/xlarge/i...,Used,2021 Chevrolet Silverado 3500 High Country,"70,989 mi.","\n $58,988\n",None,\n Fair Deal \n,None,\n White\n,\n Jet Black/Umber\n,\n Four-wheel Drive\n,\n\n0–0.0\nBased on EPA mileage ratings. Use f...,Diesel fuel type,\n Automatic\n,"\n 8cyl, 6.6L, 445 HP\n",1GC4YVEY4MF137659,38879,"70,989 mi.",\n\nAdaptive Cruise Control\nCooled Seats\nHea...,\n\nAndroid Auto®\nApple CarPlay®\nBluetooth®\...,\n\nAlloy Wheels\nSunroof/Moonroof\nTow Hitch\...,\n\nAutomatic Emergency Braking\nBackup Camera...,\n\nLeather Seats\nMemory Seat\n\n,At least 1 accident or damage reported,None,Yes,Yes,None,None,None,None,None,None,None,None,"{'Exterior color': ' White ', 'Interior ...",{'Convenience': ' Adaptive Cruise Control Coo...,"[\n120V AC power outlets\n, \n12V power outlet...",{'Accidents or damage': 'At least 1 accident o...,\n R&B Car Company Warsaw\n,4.3,\n(78 reviews)\n,"2105 Biomet Dr, Warsaw, IN 46582",https://warsawcardealers.com/?utm_source=cars....,4.2,(5 reviews)
23149,['https://platform.cstatic-images.com/xlarge/i...,New,2026 Chrysler Pacifica Limited,14 mi.,"\n $50,677\n",\n $200 price drop\n,None,None,\n PXJ\n,\n Black\n,\n Front-wheel Drive\n,\n\n19–28\nBased on EPA mileage ratings. Use f...,Gasoline,\n 9-Speed Automatic\n,\n 3.6L V6 24V MPFI DOHC\n,2C4RC1GG0TR164124,CA45,14 mi.,\n\nAdaptive Cruise Control\nHeated Seats\nHea...,\n\nApple CarPlay®\nBluetooth®\nPremium Sound ...,\n\nAlloy Wheels\nRoof Rack\nSunroof/Moonroof\n\n,\n\nBackup Camera\nBrake Assist\nRain Sensing ...,\n\nLeather Seats\nMemory Seat\nThird Row Seat...,None,None,None,None,None,None,None,None,None,None,None,None,"{'Exterior color': ' PXJ ', 'Interior co...",{'Convenience': ' Adaptive Cruise Control Hea...,"[\n13 Speakers\n, \n18 X 7.5 Painted Aluminum ...",None,\n KEY AUTO MALL - GM\n,4.5,\n(630 reviews)\n,"3700 16th St, Moline, IL 61265",https://keyautomall.com/?utm_source=cars.com&u...,NaN,None
34359,['https://platform.cstatic-images.com/xlarge/i...,New,2025 Mazda MX-5 Miata Grand Touring,5 mi.,"\n $36,041\n",None,None,None,\n Soul Red Crystal Metallic\n,\n Nk5 Black\n,\n Rear-wheel Drive\n,\n\n26–34\nBased on EPA mileage ratings. Use f...,Gasoline,\n Manual\n,\n SKYACTIV-G 2L I-4 gasoline direct inject...,JM1NDAD79S0653985,653985,5 mi.,\n\nHeated Seats\nKeyless Entry\nKeyless Start...,\n\nAndroid Auto®\nApple CarPlay®\nBluetooth®\...,\n\nAlloy Wheels\n\n,\n\nAutomatic Emergency Braking\nBackup Camera...,\n\nLeather Seats\n\n,None,None,None,None,None,None,None,None,None,None,None,None,{'Exterior color': ' Soul Red Crystal Meta...,{'Convenience': ' Heated Seats Keyless Entry ...,"[\n12V power outlets 1 12V power outlet\n, \n4...",None,\n Ocean Mazda West Kendall\n,5.0,\n(12 reviews)\n,"14000 Southwest 137th Avenue, Miami, FL 33186",https://www.oceanmazdawestkendall.com/?utm_sou...,NaN,None
47349,['https://platform.cstatic-images.com/xlarge/i...,Used,2024 Ford Transit-350 XLT,"27,568 mi.","\n $47,500\n",\n $400 price drop\n,\n Good Deal \n,None,\n OXFORD WHITE\n,\n DARK PALAZZO GRAY CLOTH\n,\n Rear-wheel Drive\n,\n\n0–0\nBased on EPA mileage ratings. Use for...,Gasoline,\n Automatic\n,"\n 3.5L V-6 port/direct injection, DOHC, va...",1FBAX2Y80RKA16579,B24923,"27,568 mi.",\n\nKeyless Entry\nNavigation System\n\n,\n\nAndroid Auto®\nApple CarPlay®\nBluetooth®\...,\n\nTow Hooks\n\n,\n\nAutomatic Emergency Braking\nBackup Camera...,\n\nThird Row Seatin

## Deep Cleaning
1. In this part we will find the way, by which we can clean our data and will combine all those code inside a function called `clean_data`
2. If the logic is a bit complex in that situation, I will create a function itself that will be written inside cleaning_helper.py file and that function will be called inside `clean_data` function.
3. Data Will be cleaned using `Method Chaining`, as Industrial relevant and optimized coding method, this way ensure that the original data remain the same, whatever changes are made will be reflected in copy of dataframe only, to avoid messy code.

#### New_used

In [162]:
(
    df['new_used']
    .str.strip()
    .str.split(' ')
    .str.get(-1)
).unique()

array(['Used', 'New', 'Certified'], dtype=object)

#### CAR NAME

In [163]:
(
    df
    .car_name
    .str.strip()
    .str.split(' ',n=2,expand=True)
    .set_axis(['model_year','Brand_name','model_name'],axis=1)
)

,model_year,Brand_name,model_name
0,2021,Chevrolet,Tahoe LT
1,2023,Mazda,CX-5 Signature
2,2023,Chevrolet,Silverado 1500 Custom
3,2026,Honda,CR-V LX AWD
4,2016,Nissan,NV200 SV
...,...,...,...
54212,2024,Toyota,Tacoma TRD Off Road
54213,2017,Dodge,Challenger GT
54214,2025,BMW,X5 xDrive40i
54215,2025,BMW,X3 30 xDrive


#### Mileage

In [164]:
# by mistake I got two col of mileage while scraping, so checking if they contain same value of not, if yes then I will drop one of them

print(df[df['mileage'] != df['Mileage']]['Mileage'].unique())
print(df[df['mileage'] != df['Mileage']]['mileage'].unique())

# both contains same values of mileage, just diff way of showing the null values hence I will drop Mileage col and keep mileage col after cleaning

(
    df
    .mileage
    .str.strip()
    .str.split()
    .str.get(0)
)

['–']
[None]


0        49,943
1        16,621
2        30,277
3             7
4        80,905
          ...  
54212    12,255
54213    18,949
54214     6,406
54215        17
54216         3
Name: mileage, Length: 54217, dtype: object

#### price

In [165]:
(
    df
    .price
    .str.strip()
    .str.replace('$','')
    .str.replace(',','')
)

0        46995
1        29999
2        34500
3        33815
4        15900
         ...  
54212    41530
54213    24366
54214    70995
54215    56260
54216    90460
Name: price, Length: 54217, dtype: object

#### price_drop

In [166]:
(
    df
    .price_drop
    .str.strip()
    .str.lower()
    .str.replace('$','')
    .str.replace('price drop','')
    .str.replace(',','')
)

0        1000 
1         354 
2         500 
3         None
4         None
         ...  
54212     459 
54213     None
54214     500 
54215     None
54216     None
Name: price_drop, Length: 54217, dtype: object

#### deal_type

In [167]:
(
    df
    .deal_type
    .str.strip()
    .str.split(' ',n=1)
    .str.get(0)
)

0         Good
1         Good
2         Fair
3         None
4         Fair
         ...  
54212     Good
54213     Good
54214     Fair
54215    Great
54216     None
Name: deal_type, Length: 54217, dtype: object

#### Exterior_color

In [168]:
(
    df
    .Exterior_color
    .str.strip()
    .replace('-',np.nan)
)

0                    Midnight Blue Metallic
1                    Rhodium White Metallic
2                                   Red Hot
3                                      Gray
4                              Fresh Powder
                        ...                
54212             Celestial Silver Metallic
54213    Granite Crystal Metallic Clearcoat
54214                Mineral White Metallic
54215               Black Sapphire Metallic
54216                     Sterling Metallic
Name: Exterior_color, Length: 54217, dtype: object

#### Interior_color

In [169]:
(
    df
    .Interior_color
    .str.strip()
    .replace('-',np.nan)
)

0                           Jet Black
1                       Caturra Brown
2                           Jet Black
3                               Black
4                                Gray
                     ...             
54212                   Boulder/Black
54213                           Black
54214                     Silverstone
54215                           Black
54216    Jet Black w/Kalahari Accents
Name: Interior_color, Length: 54217, dtype: object

#### Drivetrain

In [170]:
(
    df
    .Drivetrain
    .str.strip()
    .replace(drivetrain_map)
)

0        4WD
1        AWD
2        4WD
3        AWD
4        FWD
        ... 
54212    4WD
54213    AWD
54214    AWD
54215    AWD
54216    4WD
Name: Drivetrain, Length: 54217, dtype: object

#### MPG

In [171]:
(
    df
    .MPG
    .str.strip()
    .str.split('\n',n=1)
    .str.get(0)
    .str.split('–',expand=True)
    .set_axis(['lower_mpg','higher_mpg'],axis=1)
    .fillna('0')
    .replace('','0')
    .assign(
        lower_mpg = lambda df_ : (
            df_
            .lower_mpg
            .astype(float)
        ),

        higher_mpg = lambda df_ : (
            df_
            .higher_mpg
            .astype(float)
        )
    )
).mean(axis=1)

0        21.0
1        24.5
2        19.5
3        29.0
4        25.0
         ... 
54212    21.0
54213    22.5
54214    25.0
54215    30.0
54216     0.0
Length: 54217, dtype: float64

#### Fuel_Type

In [174]:
(
    df
    .Fuel_Type
    .replace(fuel_map)
    .unique()
)

array(['Diesel', 'Gasoline', nan, 'Hybrid', 'Other', 'Flex Fuel',
       'Plug-in Hybrid', 'CNG/LPG'], dtype=object)

#### Transmission

In [175]:
pattern = r'\b(?:\d+|ten|single|one|two|three|four|five|six|seven|eight|nine)-?\s*(?:speed|spd)\b|\b\d+A[T]?\b'

# extracting speed in transmission (in new col)
(
    df
    .Transmission
    .str.strip()
    .str.lower()
    .str.extract(f'({pattern})',flags=re.IGNORECASE)
)

# extracting transmission types
(
    df['Transmission']
    .str.strip()
    .str.lower()
    .apply(extracting_transmission).value_counts()
)

Transmission
Automatic    49875
CVT           2542
Manual        1417
DCT             65
Name: count, dtype: int64

#### Convenience

In [176]:
expending_convenience(df['Convenience'])

,adaptive_cruise_control,cooled_seats,heated_seats,heated_steering_wheels,keyless_entry,keyless_start,navigation_system,remote_start,power_liftgate,automatic_parking,power_folding_mirror,wheel_chair_accessible,autopilot
0,NaN,NaN,yes,NaN,yes,yes,yes,yes,yes,NaN,NaN,NaN,NaN
1,yes,yes,yes,yes,yes,yes,yes,yes,yes,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,yes,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN
3,yes,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54212,yes,NaN,NaN,NaN,yes,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN
54213,NaN,yes,yes,yes,yes,yes,yes,yes,NaN,NaN,NaN,NaN,NaN
54214,NaN,NaN,yes,NaN,yes,yes,yes,NaN,yes,NaN,NaN,NaN,NaN
54215,NaN,NaN,yes,yes,yes,yes,yes,yes,yes,NaN,yes,NaN,NaN


#### Entertainment

In [177]:
values = []
for i in (df.Entertainment.str.strip().str.split('\n')):
    if i is not None:
        values.extend(i)

dic = {}

for i in values:
    if i in dic:
        dic[i] += 1
    else:
        dic[i] = 1

print(dic)

{'Android Auto®': 25479, 'Apple CarPlay®': 36360, 'Bluetooth®': 50157, 'Premium Sound System': 18105, 'USB Port': 41781, 'WiFi Hotspot': 29965, 'HomeLink': 14311, 'Satellite Radio': 22877, 'CD Player': 7805, 'DVD Player': 1195, 'Rear Seat Entertainment': 929, 'Apple CarPlay®/Android Auto®': 46, 'Apple CarPlay/Android Auto': 1, 'Bluetooth': 1}


In [178]:
expanding_entertainment(df['Entertainment'])

,android_auto,apple_carplay,bluetooth,premium_sound_system,usb_port,wifi_hotspot,satellite_radio,cd_player,homelink,rear_seat_entertainment,dvd_player
0,yes,yes,yes,yes,yes,yes,NaN,NaN,NaN,NaN,NaN
1,yes,yes,yes,yes,yes,yes,yes,NaN,yes,NaN,NaN
2,yes,yes,yes,yes,yes,yes,NaN,NaN,NaN,NaN,NaN
3,yes,yes,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
54212,yes,yes,yes,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN
54213,yes,yes,yes,NaN,yes,NaN,yes,NaN,yes,NaN,NaN
54214,yes,yes,yes,yes,yes,yes,NaN,NaN,NaN,NaN,NaN
54215,yes,yes,yes,NaN,yes,yes,yes,NaN,yes,NaN,NaN


#### Exterior	

In [179]:
values = []
for i in (df.Exterior.str.strip().str.split('\n')):
    if i is not None:
        values.extend(i)

dic = {}

for i in values:
    if i in dic:
        dic[i] += 1
    else:
        dic[i] = 1

In [180]:
expanding_exterior(df['Exterior'])

,alloy_wheels,sunroof_moonroof,tow_hitch,tow_hooks,roof_rack
0,yes,yes,yes,NaN,yes
1,yes,yes,NaN,NaN,NaN
2,yes,NaN,yes,yes,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
54212,yes,NaN,yes,yes,NaN
54213,yes,NaN,NaN,NaN,NaN
54214,yes,yes,NaN,NaN,NaN
54215,yes,yes,yes,NaN,NaN


#### Safety

In [181]:
values = []
for i in (df.Safety.str.strip().str.split('\n')):
    if i is not None:
        values.extend(i)

dic = {}

for i in values:
    if i in dic:
        dic[i] += 1
    else:
        dic[i] = 1

dic

{'Automatic Emergency Braking': 38422,
 'Backup Camera': 48998,
 'Brake Assist': 50451,
 'LED Headlights': 25740,
 'Lane Departure Warning': 34310,
 'Rain Sensing Wipers': 11666,
 'Stability Control': 51396,
 'Blind Spot Monitor': 28717,
 'Rear Cross Traffic Alert': 23902}

In [182]:
expanding_safety(df['Safety'])

,automatic_emergency_braking,backup_camera,blind_spot_monitor,brake_assist,led_headlights,lane_departure_warning,rear_cross_traffic_alert,stability_control,rain_sensing_wipers
0,yes,yes,NaN,yes,yes,yes,NaN,yes,yes
1,yes,yes,yes,yes,yes,yes,yes,yes,yes
2,yes,yes,NaN,yes,NaN,yes,NaN,yes,NaN
3,yes,yes,NaN,yes,yes,yes,NaN,yes,NaN
4,NaN,NaN,NaN,yes,NaN,NaN,NaN,yes,NaN
...,...,...,...,...,...,...,...,...,...
54212,yes,yes,yes,yes,yes,yes,yes,yes,NaN
54213,yes,yes,yes,yes,NaN,NaN,yes,yes,NaN
54214,yes,yes,yes,yes,yes,yes,yes,yes,NaN
54215,yes,yes,yes,yes,yes,yes,yes,yes,yes


#### Seating

In [183]:
values = []
for i in (df.Seating.str.strip().str.split('\n')):
    if i is not None:
        values.extend(i)

dic = {}

for i in values:
    if i in dic:
        dic[i] += 1
    else:
         dic[i] = 1

dic

{'Leather Seats': 22597, 'Memory Seat': 19396, 'Third Row Seating': 8353}

In [184]:
expanding_seating(df['Seating'])

,leather_seats,memory_seat,third_row_seating
0,yes,yes,yes
1,yes,yes,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
54212,NaN,NaN,NaN
54213,yes,NaN,NaN
54214,NaN,yes,NaN
54215,NaN,yes,NaN


#### Accidents_or_damage

In [185]:
(
    df
    .Accidents_or_damage
    .value_counts()
)

Accidents_or_damage
None reported                             23512
At least 1 accident or damage reported     9990
Name: count, dtype: int64

#### Clean_title

In [186]:
(
    df
    .Clean_title
    .value_counts()
)

Clean_title
Yes    8719
No      218
Name: count, dtype: int64

#### one_owner_vehicle

In [187]:
(
    df
    .one_owner_vehicle
    .value_counts()
)

one_owner_vehicle
Yes    18663
No     14644
Name: count, dtype: int64

#### Personal_use_only

In [188]:
(
    df
    .Personal_use_only
    .value_counts()
)

Personal_use_only
Yes    22009
No     11493
Name: count, dtype: int64

#### Open_recall

In [189]:
(
    df
    .Open_recall
    .value_counts()
)

Open_recall
At least 1 open recall reported    5530
Name: count, dtype: int64

#### Fuel_Type_e

In [190]:
# this col need to be merged with Fuel_Type later.
(
    df
    .Fuel_Type_e
    .value_counts()
)

Fuel_Type_e
Electric    791
Name: count, dtype: int64

#### mpge

In [191]:
(
    df
    .mpge
    .str.split('/',expand=True)
    .set_axis(['mpge_city','mpge_hwy'],axis=1)
    .assign(
        mpge_city = lambda df_ : (
            df_
            .mpge_city
            .str.strip()
            .str.replace('\ncity','')
        ),
        mpge_hwy = lambda df_ : (
            df_
            .mpge_hwy
            .str.strip()
            .str.replace('\nhwy.','')
        )
    )
)

,mpge_city,mpge_hwy
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
54212,None,None
54213,None,None
54214,None,None
54215,None,None


#### level2_charging

In [192]:
(
    df
    .level2_charging
    .str.split('\n')
    .str.get(0)
    .str.split('-')
    .apply(lambda x : np.mean([int(i) for i in x]) if x is not None else x)
)

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
54212   NaN
54213   NaN
54214   NaN
54215   NaN
54216   NaN
Name: level2_charging, Length: 54217, dtype: float64

#### dc_fast_charging

In [193]:
(
    df
    .dc_fast_charging
    .str.strip()
    .str.replace('\nmins.','')
)

0        None
1        None
2        None
3        None
4        None
         ... 
54212    None
54213    None
54214    None
54215    None
54216    None
Name: dc_fast_charging, Length: 54217, dtype: object

#### battery_capacity

In [194]:
(
    df
    .battery_capacity
    .str.replace('\nkWh','')
    .str.split('-')
    .apply(lambda x : np.mean([int(i) for i in x]) if x is not None else x)
).unique()

array([  nan,  91. ,  72. , 131. ,  77. ,  73. ,  70. ,  60. ,  58. ,
        68. ,  98. ,  88. ,  62. ,  93. ,  40. ,  24. ,  30. ,  79.5,
        89. ,  37. ,  82. ,  27. , 123. ,  75. , 100. ,  50. ,  81. ,
        92.5,  55. ,  62.5,  16. ,  66. ,  20. , 113. ,  22. ,  18. ])

#### excepted_range

In [195]:
(
    df
    .excepted_range
    .str.replace('\nmi','')
    .str.split('-')
    .apply(lambda x : np.mean([int(i) for i in x]) if x is not None else x)
)

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
54212   NaN
54213   NaN
54214   NaN
54215   NaN
54216   NaN
Name: excepted_range, Length: 54217, dtype: float64

#### battery_range_score

In [196]:
(
    df
    .battery_range_score
    .dropna()
    .str.split('|',expand=True)
    .set_axis(['battery_range_score','battery_range_score_category'],axis=1)
    .assign(
        battery_range_score = lambda df_ : (
            df_
            .battery_range_score
            .str.strip()
        ),

        battery_range_score_category = lambda df_ : (
            df_
            .battery_range_score_category
            .str.strip()
        )
    )
)

,battery_range_score,battery_range_score_category
345,104,Outstanding
370,101,Outstanding
510,101,Outstanding
1073,103,Outstanding
1159,103,Outstanding
...,...,...
51923,103,Outstanding
52303,103,Outstanding
52661,103,Outstanding
53151,104,Outstanding


#### seller_name

In [197]:
(
    df
    .seller_name
    .str.strip()
)

0                           Lexus of Arlington
1                        DCH Ford of Eatontown
2                 AutoMax Preowned Marlborough
3        Norm Reeves Honda Superstore Cerritos
4                          Peterson Auto Sales
                         ...                  
54212                        Honda of Muskogee
54213                Covert Buick GMC Bee Cave
54214                            BMW of Darien
54215                           BMW of Fremont
54216           Corwin Buick GMC Cadillac Reno
Name: seller_name, Length: 54217, dtype: object

#### seller_rating

In [198]:
(
    df
    .seller_rating
    .unique()
)

array([4.9, 4.8, 5. , 0. , 4.6, 4.5, 4.7, 2.2, 3.2, 3.6, 4.1, 3. , 4.3,
       2.4, 4.2, 4.4, 2.9, 4. , 3.7, 3.1, 2.8, 3.3, 2.6, 2.7, 1.7, 3.9,
       1.6, 1.9, 3.8, 3.5, 2.1, 1.5, 2. , 1.4, 2.5, 3.4, 1.8, 1.1, 1.3,
       1.2, 1. , 2.3])

#### people_count_seller_rating

In [199]:
(
    df
    .people_count_seller_rating
    .str.replace('(','')
    .str.replace('reviews)','')
    .str.replace('review)','')
    .str.replace(',','')
    .str.strip()
)

0         426
1         661
2         194
3         669
4          22
         ... 
54212      21
54213      12
54214      50
54215    1873
54216     305
Name: people_count_seller_rating, Length: 54217, dtype: object

#### seller_address

In [200]:
expanding_seller_address(df['seller_address'])

,street_address,city,state,postal_code
0,1510 W Dundee Rd,Palatine,IL,60074
1,85 NJ-36,Eatontown,NJ,07724
2,400 Maple St. Rt. 85,Marlborough,MA,01752
3,18500 Studebaker Rd.,Cerritos,CA,90703
4,8498 S. State St,Midvale,UT,84047
...,...,...,...,...
54212,240 West Shawnee Street,Muskogee,OK,74401
54213,16501 Sweetwater Village Dr,Austin,TX,78738
54214,140 Ledge Rd,Darien,CT,06820
54215,5720 Cushing Parkway,Fremont,CA,94538


#### seller_site

In [201]:
(
    df
    .seller_site
    .value_counts()
)

seller_site
https://gatewayclassiccars.com/?utm_source=cars.com&utm_medium=referral&utm_campaign=cars.com_VDP_referral                    74
https://www.mountainstatestoyota.com/?utm_source=cars.com&utm_medium=referral&utm_campaign=cars.com_VDP_referral              56
http://www.planetford45.com?utm_source=cars.com&utm_medium=referral&utm_campaign=cars.com_VDP_referral                        55
http://www.normreeveshondacerritos.com?utm_source=cars.com&utm_medium=referral&utm_campaign=cars.com_VDP_referral             55
http://www.floridafinecars.com?utm_source=cars.com&utm_medium=referral&utm_campaign=cars.com_VDP_referral                     54
                                                                                                                              ..
http://www.worldimportsusa.com?utm_source=cars.com&utm_medium=referral&utm_campaign=cars.com_VDP_referral                      1
https://www.equipautosales.com/?utm_source=cars.com&utm_medium=referral&utm_campaign=

#### car_rating

In [202]:
(
    df
    .car_rating
)

0        4.1
1        4.3
2        3.8
3        3.7
4        5.0
        ... 
54212    4.2
54213    4.8
54214    4.2
54215    4.0
54216    NaN
Name: car_rating, Length: 54217, dtype: float64

#### people_count_car_rating

In [203]:
(
    df
    .people_count_car_rating
    .str.extract(r'(\d+)')
)

,0
0,47
1,13
2,8
3,3
4,5
...,...
54212,8
54213,172
54214,5
54215,2


#### Engine

In [204]:
# engine_size col
(
    df
    .Engine
    .str
    .extract(r'(?i)(\d+(?:\.\d+)?[-\s]*(?:L|liter))',flags=re.IGNORECASE)
)

,0
0,3L
1,2.5L
2,2.7L
3,1.5L
4,2L
...,...
54212,2.4L
54213,3.6L
54214,3L
54215,2L


In [205]:
# cylinder_config
(
    df
    .Engine
    .str
    .extract(r'([IVH]\s*-?\s*\d{1,2})')
)

,0
0,I-6
1,I-4
2,I-4
3,I-4
4,I-4
...,...
54212,I-4
54213,V-6
54214,I-6
54215,I-4


In [206]:
# VALVES
(
    df
    .Engine
    .str.extract(r"\b(\d{1,2})\s*[-]?\s*(?:V|Valve)\b")
)

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
54212,NaN
54213,NaN
54214,NaN
54215,NaN


In [207]:
def clean_data(df):
    """
    This Function cars dataframe after level-1 cleaning and 
    do relevent operation on each column that will either clean or clean as well as expand the column if needed

    returns a level-2 cleaned dataframe
    """
    return ((
                expanding_car_name_col(df['car_name'])).join(
                df
                .assign(
                    new_used = lambda df_ : (
                        df_
                        .new_used
                        .str.strip()
                        .str.split(' ')
                        .str.get(-1)
                    ),

                    mileage = lambda df_ : (
                            df_
                            .mileage
                            .str.strip()
                            .str.split()
                            .str.get(0)
                    ),

                    price = lambda df_ : (
                            df_
                            .price
                            .str.strip()
                            .str.replace('$','')
                            .str.replace(',','')
                    ),

                    price_drop = lambda df_ : (
                                df_
                                .price_drop
                                .str.strip()
                                .str.lower()
                                .str.replace('$','')
                                .str.replace('price drop','')
                                .str.replace(',','')
                    ),

                    deal_type = lambda df_ : (
                            df_
                            .deal_type
                            .str.strip()
                            .str.split(' ',n=1)
                            .str.get(0)
                    ),

                    Exterior_color = lambda df_ : (
                            df_
                            .Exterior_color
                            .str.strip()
                            .replace('-',np.nan)
                    ),

                    Interior_color = lambda df_ : (
                            df_
                            .Interior_color
                            .str.strip()
                            .replace('-',np.nan)
                    ),

                    Drivetrain = lambda df_ : (
                        df_
                        .Drivetrain
                        .str.strip()
                        .replace(drivetrain_map)
                    ),

                    MPG = lambda df_ : (
                        df_
                        .MPG
                        .pipe(MPG_mean_calculation)
                    ),

                    Fuel_Type = lambda df_ : (
                        df_
                        .Fuel_Type
                        .replace(fuel_map)
                    ),

                    gear_spec = lambda df_ : (
                        df_
                        .Transmission
                        .pipe(speeds_in_transmission)
                    ),

                    Transmission = lambda df_ : (
                        df_
                        .Transmission
                        .apply(extracting_transmission)
                    ),

                    level2_charging = lambda df_ : (
                                df_
                                .level2_charging
                                .str.split('\n')
                                .str.get(0)
                                .str.split('-')
                                .apply(lambda x : np.mean([int(i) for i in x]) if x is not None else x)
                    ),

                    dc_fast_charging = lambda df_ : (
                            df_
                            .dc_fast_charging
                            .str.strip()
                            .str.replace('\nmins.','') 
                    ),

                    battery_capacity = lambda df_ : (
                            df_
                            .battery_capacity
                            .str.replace('\nkWh','')
                            .str.split('-')
                            .apply(lambda x : np.mean([int(i) for i in x]) if x is not None else x)
                    ),

                    excepted_range  = lambda df_ : (
                        df_
                        .excepted_range
                        .str.replace('\nmi','')
                        .str.split('-')
                        .apply(lambda x : np.mean([int(i) for i in x]) if x is not None else x)
                    ),

                    seller_name = lambda df_ : (
                            df_
                            .seller_name
                            .str.strip()
                    ),

                    people_count_seller_rating = lambda df_ : (
                    df_
                    .people_count_seller_rating
                    .str.replace('(','')
                    .str.replace('reviews)','')
                    .str.replace('review)','')
                    .str.replace(',','')
                    .str.strip()
                    ),

                    people_count_car_rating  = lambda df_ :  (
                            df_
                            .people_count_car_rating
                            .str.extract(r'(\d+)')
                    ),

                    engine_size = lambda df_ : (
                            df_
                            .Engine
                            .str
                            .extract(r'(?i)(\d+(?:\.\d+)?[-\s]*(?:L|liter))',flags=re.IGNORECASE)
                    ),

                    cylinder_config = lambda df_ : (
                        df_
                        .Engine
                        .str
                        .extract(r'([IVH]\s*-?\s*\d{1,2})')
                    ),

                    valves = lambda df_ :  (
                        df_
                        .Engine
                        .str.extract(r"\b(\d{1,2})\s*[-]?\s*(?:V|Valve)\b")
                    )
                )
            )
            .join(expending_convenience(df['Convenience']))
            .join(expanding_entertainment(df['Entertainment']))
            .join(expanding_exterior(df['Exterior']))
            .join(expanding_safety(df['Safety']))
            .join(expanding_seating(df['Seating']))
            .join(expanding_mpge(df['mpge']))
            .join(expanding_battery_range_score(df['battery_range_score']))
            .join(expanding_seller_address(df['seller_address']))
            .drop(columns=['car_name','key_specs','basics','features','other_features','vehicle_history','Mileage','Convenience','Entertainment','VIN','Stock_no','Exterior','Safety','Seating','mpge','seller_address','Engine','battery_range_score'])
        )

data = clean_data(df)

In [208]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54217 entries, 0 to 54216
Data columns (total 84 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   model_year                    54217 non-null  object 
 1   Brand_name                    54217 non-null  object 
 2   model_name                    54217 non-null  object 
 3   image_list                    53320 non-null  object 
 4   new_used                      54217 non-null  object 
 5   mileage                       53498 non-null  object 
 6   price                         54217 non-null  object 
 7   price_drop                    22257 non-null  object 
 8   deal_type                     32589 non-null  object 
 9   Exterior_color                54213 non-null  object 
 10  Interior_color                54211 non-null  object 
 11  Drivetrain                    53384 non-null  object 
 12  MPG                           54217 non-null  float64
 13  F

In [210]:
# exporting data to parquet file

data.to_parquet('POST_CLEANING_2_DATA_PART_5.parquet',engine='fastparquet')